In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

Append the `grt_reader` repo so we can read modules from sibling directories.


In [3]:
sys.path.append(os.path.abspath(os.path.join("../..")))

In [4]:
import numpy as np
import pandas as pd

from notebooks.utils.constants import (
    AvailableEmbeddingModels,
    AvailableModels,
    question_ids_map,
)
from notebooks.utils.process_lease import process_lease
from notebooks.utils.questions import (
    construct_single_question_for_ai,
)

/Users/david/eafit/eafit-tesis/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Retrieve questions


In [5]:
questions_df = pd.read_excel("../../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [6]:
answers_df = pd.read_excel("../../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [7]:
question_id = question_ids_map["commencement_date"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

In [8]:
file_name = "lease001"
local_file = f"../../data/asc_842/lease_agreements/{file_name}.pdf"
file_key = f"eafit/{file_name}.pdf"
namespace = f"eafit_{file_name}"

In [9]:
model: AvailableModels = "gpt-3.5-turbo"
embedding_model: AvailableEmbeddingModels = "text-embedding-3-large"

In [10]:
result = process_lease(
    file_name=file_name,
    namespace=namespace,
    question_for_ai=question_for_ai,
    question_id=question_id,
    answers_df=answers_df,
    model=model,
    embedding_model=embedding_model,
    index_name="eafit-text-embedding-3-large",
)

In [11]:
answer = result["answer"]
real_answer_unprocessed = result["real_answer_unprocessed"]

In [12]:
formatted_answer = real_answer_unprocessed.iloc[0].strftime("%Y-%m-%d")

In [13]:
formatted_answer == answer

False